In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import sklearn as sk
import random
import math as mat
#df.info()

In [4]:
#блок функций
def rmse(y, y_pred):
    return mat.sqrt(((y-y_pred)**2 ).sum()/len(y_pred)) 
def get_r2(x, y):
    slope, intercept = np.polyfit(x, y, 1)
    r_squared = 1 - (sum((y - (slope * x + intercept))**2) / ((len(y) - 1) * np.var(y, ddof=1)))
    return r_squared
#проверка типов фич
def is_floating(x):
    return (x - x.astype(int)).abs().sum()>0

def is_boolean(x):
    return x.unique().shape[0] == 2

def is_const(x):
    return x.unique().shape[0] == 1

#pd.DataFrame({
    #'float': df.apply(is_floating, axis=0),
    #'boolean': df.apply(is_boolean, axis=0),
    #'const': df.apply(is_const, axis=0)
    
#})
#проверка корреляции (по Пирсону) фич
def is_cor(X):
    iscor = X.corr()['Target'][X.corr()['Target'] < 1].abs()
    iscor = iscor.sort_values(ascending=False)
    return iscor

In [5]:
#чтение данных
path = r'D:\Bugaychenko\Firstask\dataset1\dataset.csv'#
df = pd.read_csv(path)

#удаление лишних столбцов
df = df.drop('Post Promotion Status', 1)
df = df.drop('Unnamed: 0', 1)
df = df.sample(frac=1)  #перемещиваем

#нормировка кроме таргета и булевых
for c in df.columns: 
    if((c!='Target') & (is_boolean(df[c])== False)):
        mean, std = df[c].mean(), df[c].std() 
        df[c]=(df[c] - mean)/std 
df.to_csv(r'D:\Bugaychenko\Firstask\dataset1\datasetOUT.csv')
#df[df.columns[52]]
#df[df.columns[52]].to_excel(r'C:\Users\SUB\Desktop\test folder\dataout.xlsx', startrow=3, index=False)# выгрузка в эксель

In [26]:
#проверка удаления таргетов
#for i in range(1,100):
    #print(i, len(df[df.Target > i])/40949)
#df = df.drop(df[df.Target > 100].index) #удаление Таргет>100
print(df.columns[0:52].values)
#.to_excel(r'C:\Users\SUB\Desktop\test folder\dataout.xlsx', startrow=3, index=False)# выгрузка в эксель

['Page Popularity' 'Page Checkins' 'Page talking about' 'Page Category'
 'extra_0' 'extra_1' 'extra_2' 'extra_3' 'extra_4' 'extra_5' 'extra_6'
 'extra_7' 'extra_8' 'extra_9' 'extra_10' 'extra_11' 'extra_12' 'extra_13'
 'extra_14' 'extra_15' 'extra_16' 'extra_17' 'extra_18' 'extra_19'
 'extra_20' 'extra_21' 'extra_22' 'extra_23' 'extra_24' 'CC1' 'CC2' 'CC3'
 'CC4' 'CC5' 'Base Time' 'Post Length' 'Post Share Count' 'H Local'
 'published_weekday_0' 'published_weekday_1' 'published_weekday_2'
 'published_weekday_3' 'published_weekday_4' 'published_weekday_5'
 'published_weekday_6' 'base_weekday_0' 'base_weekday_1' 'base_weekday_2'
 'base_weekday_3' 'base_weekday_4' 'base_weekday_5' 'base_weekday_6']


In [6]:
df=pd.read_csv(r'D:\Bugaychenko\Firstask\dataset1\datasetOUT.csv')
df = df.drop('Unnamed: 0', 1)
#df = df.drop(df[df.Target > 100].index)
fold = np.array_split(df, 5)
iscor=is_cor(df)#корреляция столбцов
#print(iscor)

In [10]:
#библиотека sklearn

RMSE_sum=0
R2_sum=0
RMSE_sumTEST=0
R2_sumTEST=0
for i in range(0,5):
    pdcont=df.drop(fold[4-i].index)
    features = df.columns[0:52]
    training_input = pdcont[features].values
    training_output = pdcont['Target'].values
    test_input = fold[4-i][features].values
    test_output = fold[4-i]['Target'].values
    
    linreg = LinearRegression().fit(training_input,training_output)
    #print("REGscore:",linreg.score(test_input,test_output))
    #print(linreg.coef_)
    
    print("TEST FOLD      ", i)
    print("RMSE:",rmse(test_output,linreg.predict(test_input)))    
    print("R2score:",get_r2(test_output,linreg.predict(test_input)))

    print("TRAIN FOLD      ", i)
    print("RMSE:",rmse(training_output,linreg.predict(training_input)))         
    print("R2score:",get_r2(training_output,linreg.predict(training_input)))

TEST FOLD       0
RMSE: 25.943238226383798
R2score: 0.3163577046440388
TRAIN FOLD       0
RMSE: 29.94555147303302
R2score: 0.32564192269228387
TEST FOLD       1
RMSE: 30.82504197711158
R2score: 0.3042656703129716
TRAIN FOLD       1
RMSE: 28.762575815273784
R2score: 0.329476712665025
TEST FOLD       2
RMSE: 29.261815853243117
R2score: 0.32709998779548255
TRAIN FOLD       2
RMSE: 29.149654449678458
R2score: 0.32387148574717906
TEST FOLD       3
RMSE: 28.77733186971606
R2score: 0.27619896184272397
TRAIN FOLD       3
RMSE: 29.541077129077
R2score: 0.32722529582466375
TEST FOLD       4
RMSE: 32.978302439793126
R2score: 0.311044198897908
TRAIN FOLD       4
RMSE: 28.160820113070983
R2score: 0.3289594214331717


In [28]:
def normal_equation(X, y): #вычисляет вектор весов w согласно нормальному уравнению линейной регрессии.
    a = np.dot(X.T, X) 
    b = np.dot(X.T, y) 
    res = np.linalg.solve(a, b) 
    return res
features = df.columns[0:52]

def linear_prediction(X, w): #предсказание
    return np.dot(X, w)

def gradient_step_v(X, y, w, train_ind, eta=0.01): #шаг градиента
    N = X.shape[0]                   
    x = X[train_ind]                 
    y_pred = linear_prediction(x, w) 
    rs = (y_pred - y[train_ind])     
    return  w - 2.0* eta/N*x*rs

def gradient_descent(X, y, w = 0, eta=0.2, max_iter= df.shape[0],epsilon=1e-5): #градиентный спуск
    iter_num = 0
    max_iter = X.shape[0]
    w = gradient_step_v(X, y, w, iter_num, eta)
    while iter_num < max_iter:
        eta-=1e-6
        w  = gradient_step_v(X, y, w, iter_num, eta) 
        iter_num += 1
    return w    
    



#stoch_grad_desc_weights = gradient_descent(df[features].values, df['Target'].values)             
#print(stoch_grad_desc_weights)
#print(rmse(df['Target'].values, linear_prediction(df[features].values, stoch_grad_desc_weights)))
#print(get_r2(df['Target'].values,linear_prediction(df[features].values, stoch_grad_desc_weights)))

In [29]:
r2_test=[]
r2_train=[]
rmse_test=[]
rmse_train=[]
feat = pd.DataFrame(np.zeros((52, 5)),index=[df.columns[0:52]])
for i in range(0,5):
    pdcont=df.drop(fold[4-i].index)
    features = df.columns[0:52]
    training_input = pdcont[features].values
    training_output = pdcont['Target'].values
    test_input = fold[4-i][features].values
    test_output = fold[4-i]['Target'].values
    
    #norm_eq_w= normal_equation(training_input, training_output)
    linreg = gradient_descent(training_input,training_output)#oбучаем
    #r2 test train
    r2_test.append(get_r2(test_output,linear_prediction(test_input, linreg)))
    r2_train.append(get_r2(training_output,linear_prediction(training_input, linreg)))
   # 
    rmse_test.append(rmse(test_output,linear_prediction(test_input, linreg)))
    rmse_train.append(rmse(training_output,linear_prediction(training_input, linreg)))
#     colum = [r2_test, r2_train, rmse_test, rmse_train] + linreg.tolist()
#     fine[fine.columns[i]] = colum
# fine.to_excel(r'C:\Users\SUB\Desktop\test folder\dataout.xlsx', index=True)
    feat[i]=linreg
#     print("TEST FOLD      ", i)
#     print("RMSE:",rmse(test_output,linear_prediction(test_input, linreg)))
#     RMSE_sum += rmse(test_output,linear_prediction(test_input, linreg))           
#     print("R2score:",get_r2(test_output,linear_prediction(test_input, linreg)))
#     R2_sum +=get_r2(test_output,linear_prediction(test_input, linreg))  

#     print("TRAIN FOLD      ", i)
#     print("RMSE:",rmse(training_output,linear_prediction(training_input, linreg)))          
#     print("R2score:",get_r2(training_output,linear_prediction(training_input, linreg)))
# print("R2 mean",R2_sum/5)
# print("RMSE mean",RMSE_sum/5)

In [30]:
pd.options.display.expand_frame_repr = False
df1 = pd.DataFrame(np.vstack([r2_test, r2_train, rmse_test, rmse_train]), index=[
    'r2_test',
    'r2_train',
    'rmse_test',
    'rmse_train',
] )
df1 = pd.concat([df1,feat],axis=0)
df1 = pd.concat([df1, df1.mean(axis=1).rename('mean'), df1.std(axis=1).rename('std')], axis=1)
print(df1)



                                0          1          2          3          4       mean       std
r2_test                  0.262321   0.268467   0.295327   0.304838   0.275592   0.281309  0.018085
r2_train                 0.280431   0.285495   0.281431   0.282070   0.286985   0.283282  0.002812
rmse_test               27.733105  32.647502  31.242063  28.944728  35.236917  31.160863  2.978745
rmse_train              31.979170  30.892370  31.247726  31.723936  30.176891  31.204019  0.711547
(Page Popularity,)       0.034315  -0.002493  -0.001435  -0.056486  -0.065563  -0.018332  0.041817
(Page Checkins,)        -0.253365  -0.206506  -0.253097  -0.242052  -0.238784  -0.238761  0.019170
(Page talking about,)    0.169544   0.059861   0.141514   0.142586   0.105927   0.123886  0.042328
(Page Category,)         0.015330  -0.024826  -0.006780  -0.006007   0.008259  -0.002805  0.015510
(extra_0,)               0.363999   0.349825   0.254853   0.264892   0.208159   0.288346  0.066342
(extra_1,)